In [2]:
pip install --no-cache-dir numpy==1.26.4 pandas==2.2.2 scikit-learn==1.5.1

     ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/15.8 MB 7.9 MB/s eta 0:00:02
     ------- -------------------------------- 2.9/15.8 MB 8.7 MB/s eta 0:00:02
     ------------- -------------------------- 5.2/15.8 MB 9.6 MB/s eta 0:00:02
     ------------------ --------------------- 7.3/15.8 MB 10.0 MB/s eta 0:00:01
     ------------------------ --------------- 9.7/15.8 MB 10.2 MB/s eta 0:00:01
     ----------------------------- --------- 12.1/15.8 MB 10.4 MB/s eta 0:00:01
     ----------------------------------- --- 14.4/15.8 MB 10.6 MB/s eta 0:00:01
     ---------------------------------------- 15.8/15.8 MB 10.7 MB/s  0:00:01
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies:

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      + c:\Users\mudav\Desktop\web development\.venv\Scripts\python.exe C:\Users\mudav\AppData\Local\Temp\pip-install-we5si9no\numpy_b6e7527b481c479ea28192663d255778\vendored-meson\meson\meson.py setup C:\Users\mudav\AppData\Local\Temp\pip-install-we5si9no\numpy_b6e7527b481c479ea28192663d255778 C:\Users\mudav\AppData\Local\Temp\pip-install-we5si9no\numpy_b6e7527b481c479ea28192663d255778\.mesonpy-re_uk085 -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\mudav\AppData\Local\Temp\pip-install-we5si9no\numpy_b6e7527b481c479ea28192663d255778\.mesonpy-re_uk085\meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: C:\Users\mudav\AppData\Local\Temp\pip-install-we5si9no\numpy_b6e7527b481c479ea28192663d255778
      Build dir: C:\Users\mudav\AppData\Local\Temp\pip-install-we5si

In [3]:
!pip install --upgrade pip setuptools wheel

In [6]:
pip install numpy==1.26.4 pandas==2.2.2 scikit-learn==1.4.2 joblib flask gunicorn

  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      + c:\Users\mudav\Desktop\web development\.venv\Scripts\python.exe C:\Users\mudav\AppData\Local\Temp\pip-install-tj5e9gqg\numpy_361d00b9384445ff90c42a1d060b43f1\vendored-meson\meson\meson.py setup C:\Users\mudav\AppData\Local\Temp\pip-install-tj5e9gqg\numpy_361d00b9384445ff90c42a1d060b43f1 C:\Users\mudav\AppData\Local\Temp\pip-install-tj5e9gqg\numpy_361d00b9384445ff90c42a1d060b43f1\.mesonpy-5gn_mcp4 -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\mudav\AppData\Local\Temp\pip-install-tj5e9gqg\numpy_361d00b9384445ff90c42a1d060b43f1\.mesonpy-5gn_mcp4\meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: C:\Users\mudav\AppData\Local\Temp\pip-install-tj5e9gqg\numpy_361d00b9384445ff90c42a1d060b43f1
      Build dir: C:\Users\mudav\AppData\Local\Temp\pip-install-tj5e9

In [8]:
pip install sagemaker==2.219.0 boto3 pandas scikit-learn==1.5.1


  Using cached sagemaker-2.219.0-py3-none-any.whl.metadata (14 kB)
  Using cached boto3-1.40.11-py3-none-any.whl.metadata (6.7 kB)
  Using cached scikit_learn-1.5.1.tar.gz (7.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      + meson setup C:\Users\mudav\AppData\Local\Temp\pip-install-ilcdhls3\scikit-learn_1d78d279e4df449f83f26a29a3871fd4 C:\Users\mudav\AppData\Local\Temp\pip-install-ilcdhls3\scikit-learn_1d78d279e4df449f83f26a29a3871fd4\.mesonpy-lue8389w -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\mudav\AppData\Local\Temp\pip-install-ilcdhls3\scikit-learn_1d78d279e4df449f83f26a29a3871fd4\.mesonpy-lue8389w\meson-python-native-file.ini
      The Meson build system
      Version: 1.8.3
      Source dir: C:\Users\mudav\AppData\Local\Temp\pip-install-ilcdhls3\scikit-learn_1d78d279e4df449f83f26a29a3871fd4
      Build dir: C:\Users\mudav\AppData\Local\Temp\pip-install-ilcdhls3\scikit-learn_1d78d279e4df449f83f26a29a3871fd4\.mesonpy-lue8389w
      Build type: native build
      Project name: scikit-learn
      Project version: 

In [13]:
pip install sagemaker boto3 pandas numpy


  Using cached sagemaker-2.250.0-py3-none-any.whl.metadata (17 kB)
  Using cached boto3-1.40.11-py3-none-any.whl.metadata (6.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached fastapi-0.116.1-py3-none-any.whl.metadata (28 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached importlib_metadata-6.11.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached jsonschema-4.25.0-py3-none-any.whl.metadata (7.7 kB)
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend d

  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      + c:\Users\mudav\Desktop\web development\.venv\Scripts\python.exe C:\Users\mudav\AppData\Local\Temp\pip-install-312xlk_j\numpy_2a2935178ae04ebca669ffdbca713f32\vendored-meson\meson\meson.py setup C:\Users\mudav\AppData\Local\Temp\pip-install-312xlk_j\numpy_2a2935178ae04ebca669ffdbca713f32 C:\Users\mudav\AppData\Local\Temp\pip-install-312xlk_j\numpy_2a2935178ae04ebca669ffdbca713f32\.mesonpy-v2fyufls -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\mudav\AppData\Local\Temp\pip-install-312xlk_j\numpy_2a2935178ae04ebca669ffdbca713f32\.mesonpy-v2fyufls\meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: C:\Users\mudav\AppData\Local\Temp\pip-install-312xlk_j\numpy_2a2935178ae04ebca669ffdbca713f32
      Build dir: C:\Users\mudav\AppData\Local\Temp\pip-install-312xl

In [19]:
python -m pip install --upgrade pip setuptools wheel

# Uninstall broken scikit-learn if partially installed
pip uninstall -y scikit-learn

# Now install scikit-learn from prebuilt wheels
pip install scikit-learn==1.5.1 --only-binary :all:


SyntaxError: invalid syntax (4013889023.py, line 1)

In [18]:
# Environment setup
!pip install sagemaker==2.219.0 boto3 pandas scikit-learn==1.5.1 joblib

import sagemaker, boto3, pandas as pd, numpy as np, os, json
from sagemaker.sklearn import SKLearn
from sagemaker import get_execution_role

session = sagemaker.Session()
region = session.boto_region_name
s3 = boto3.client("s3", region_name=region)

try:
    role = get_execution_role()
except Exception:
    role = "arn:aws:iam::YOUR_ACCOUNT_ID:role/service-role/AmazonSageMaker-ExecutionRole"

bucket = session.default_bucket()
prefix = "wine-quality-linear"

print("Region:", region)
print("Bucket:", bucket)
print("Role:", role)


  Using cached sagemaker-2.219.0-py3-none-any.whl.metadata (14 kB)
  Using cached boto3-1.40.11-py3-none-any.whl.metadata (6.7 kB)
  Using cached scikit_learn-1.5.1.tar.gz (7.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      + meson setup C:\Users\mudav\AppData\Local\Temp\pip-install-1c25q91z\scikit-learn_b7aab7e14b024c3b9d10cb410310f596 C:\Users\mudav\AppData\Local\Temp\pip-install-1c25q91z\scikit-learn_b7aab7e14b024c3b9d10cb410310f596\.mesonpy-n1l2fgjc -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\mudav\AppData\Local\Temp\pip-install-1c25q91z\scikit-learn_b7aab7e14b024c3b9d10cb410310f596\.mesonpy-n1l2fgjc\meson-python-native-file.ini
      The Meson build system
      Version: 1.8.3
      Source dir: C:\Users\mudav\AppData\Local\Temp\pip-install-1c25q91z\scikit-learn_b7aab7e14b024c3b9d10cb410310f596
      Build dir: C:\Users\mudav\AppData\Local\Temp\pip-install-1c25q91z\scikit-learn_b7aab7e14b024c3b9d10cb410310f596\.mesonpy-n1l2fgjc
      Build type: native build
      Project name: scikit-learn
      Project version: 

ModuleNotFoundError: No module named 'sagemaker'